In [ ]:
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import accuracy_score

In [ ]:
from google.colab import drive
from IPython.display import Image, display

mount_point = "/content/gdrive"
base_path = mount_point + "/MyDrive/Colab/SDG/data"

drive.mount(mount_point, force_remount=True)

Mounted at /content/gdrive


In [ ]:
df = pd.read_excel(base_path + "/../AnalisiTesti/data/output/RisultatiSentiment_5modelli_CON_LABEL_FINALE.xlsx")
df

,Unnamed: 0.1,Unnamed: 0,text_id,text,sdg_true,sentiment_Llama3,sentiment_Mixtral,sentiment_Phi3,sentiment_Gemma,sentiment_Qwen,vote_level,label
0,0,0,1528,"Indeed, available empirical evidence typically...",10,neutral,neutral,negative,negative,negative,Silver,negative
1,1,1,2389,Having a secure and up-to-date knowledge datab...,15,neutral,positive,positive,positive,positive,Gold,positive
2,2,2,5801,It may be commonplace to state that the world ...,16,neutral,neutral,neutral,positive,negative,Silver,neutral
3,3,3,420,From the environmental protection point of vie...,12,positive,positive,positive,positive,positive,Platinum,positive
4,4,4,5138,"In other countries, such as the Czech Republic...",10,neutral,neutral,positive,positive,positive,Silver,positive
...,...,...,...,...,...,...,...,...,...,...,...,...
6395,6395,6395,2870,1. What's morality got to do with it? making t...,16,neutral,neutral,neutral,neutral,neutral,Platinum,neutral
6396,6396,6396,6330,As an organising principle within multi-level ...,16,neutral,neutral,neutral,positive,neutral,Gold,neutral
6397,6397,6397,2723,This article examines the neo-conservative cri...,16,neutral,neutral,neutral,negative,negative,Silver,neutral
6398,6398,6398,6710,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,neutral,neutral,neutral,positive,neutral,Gold,neutral


In [ ]:
df['text'] = 'SDG-' + df['sdg_true'].astype(str) + ' --- ' + df['text']

In [ ]:
df['text'].iloc[1]

'SDG-15 --- Having a secure and up-to-date knowledge database is, however, crucial to any effective conservation strategy. The profile also identified 17 important corridors within the Mediterranean Basin Biodiversity Hotspot, five of which traverse areas in Morocco (table 9.3). Of these, six have been created since 2003: the Al Hoceima National Park (established in 2004), Talassemtane National Park (2004), Ifrane National Park (2004), High Atlas Oriental National Park (2004), Khenifiss National Park (2006) and Khenifra National Park (2008).'

In [ ]:
# BISOGNA RIMUOVERE DAL DATASET I TESTI UTILIZZATI PER L'ETICHETTATURA MANUALE DEL DATASET DI TEST
test_df = pd.read_excel(base_path + "/../AnalisiTesti/data/output/RisultatiSentiment_EtichetatturaManuale_DatasetDiTest_CON_LABEL_FINALE.xlsx").drop(columns=['Unnamed: 0'])
test_df['text'] = 'SDG-' + test_df['sdg_true'].astype(str) + ' --- ' + test_df['text']

# test_df['text'] = test_df['text'].str.lower() # PERCHE' USIAMO BERT BASE UNCASED
# Convertire manualmente il testo potrebbe essere ridondante e, in alcuni casi, rischiare di introdurre errori o incoerenze se non si utilizza esattamente lo stesso schema di pre-processing previsto dal tokenizer.
# Il tokenizer di BERT uncased gestisce automaticamente la conversione dei caratteri in minuscolo, garantendo coerenza con il pre-addestramento.
# per un task di classificazione testi utilizzando BERT base uncased, è consigliabile utilizzare il tokenizer fornito (ad esempio, tramite la libreria Hugging Face Transformers) e lasciargli gestire la normalizzazione del testo, inclusa la conversione in minuscolo.

test_df['label'] = test_df['label'].apply(lambda x: 2 if x == 'negative' else 0 if x == 'neutral' else 1)
test_df['label'] = test_df['label'].astype(int)
test_df

,text_id,text,sdg_true,label
0,210,SDG-1 --- The Brazilian Instituto de Pesquisa ...,1,1
1,251,SDG-1 --- While the growth effect predominated...,1,0
2,441,"SDG-1 --- Nevertheless, the use of indices as ...",1,1
3,591,"SDG-1 --- If it is high, then public policies ...",1,1
4,1250,SDG-1 --- Results for countries using national...,1,0
...,...,...,...,...
571,5801,SDG-16 --- It may be commonplace to state that...,16,0
572,6119,SDG-16 --- Sexual citizenship and sexual right...,16,1
573,6551,SDG-16 --- Legal pluralism is a phenomenon def...,16,1
574,6618,"SDG-16 --- This article discusses the extent, ...",16,2


In [ ]:
# BISOGNA RIMUOVERE DAL DATASET I TESTI UTILIZZATI PER L'ETICHETTATURA MANUALE DEL DATASET DI TEST
train_df = df[~df.text_id.isin(test_df.text_id)]

# train_df['text'] = train_df['text'].str.lower() # PERCHE' USIAMO BERT BASE UNCASED
# Convertire manualmente il testo potrebbe essere ridondante e, in alcuni casi, rischiare di introdurre errori o incoerenze se non si utilizza esattamente lo stesso schema di pre-processing previsto dal tokenizer.
# Il tokenizer di BERT uncased gestisce automaticamente la conversione dei caratteri in minuscolo, garantendo coerenza con il pre-addestramento.
# per un task di classificazione testi utilizzando BERT base uncased, è consigliabile utilizzare il tokenizer fornito (ad esempio, tramite la libreria Hugging Face Transformers) e lasciargli gestire la normalizzazione del testo, inclusa la conversione in minuscolo.

train_df['label'] = train_df['label'].apply(lambda x: 2 if x == 'negative' else 0 if x == 'neutral' else 1)
train_df['label'] = train_df['label'].astype(int)
train_df

<ipython-input-7-28b16fd64c9f>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['label'] = train_df['label'].apply(lambda x: 2 if x == 'negative' else 0 if x == 'neutral' else 1)
<ipython-input-7-28b16fd64c9f>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['label'] = train_df['label'].astype(int)


,Unnamed: 0.1,Unnamed: 0,text_id,text,sdg_true,sentiment_Llama3,sentiment_Mixtral,sentiment_Phi3,sentiment_Gemma,sentiment_Qwen,vote_level,label
0,0,0,1528,"SDG-10 --- Indeed, available empirical evidenc...",10,neutral,neutral,negative,negative,negative,Silver,2
1,1,1,2389,SDG-15 --- Having a secure and up-to-date know...,15,neutral,positive,positive,positive,positive,Gold,1
3,3,3,420,SDG-12 --- From the environmental protection p...,12,positive,positive,positive,positive,positive,Platinum,1
4,4,4,5138,"SDG-10 --- In other countries, such as the Cze...",10,neutral,neutral,positive,positive,positive,Silver,1
5,5,5,3504,SDG-7 --- They also show causality in both dir...,7,positive,positive,positive,positive,positive,Platinum,1
...,...,...,...,...,...,...,...,...,...,...,...,...
6394,6394,6394,2370,SDG-16 --- Purpose – This article aims to expl...,16,positive,neutral,positive,positive,positive,Gold,1
6396,6396,6396,6330,SDG-16 --- As an organising principle within m...,16,neutral,neutral,neutral,positive,neutral,Gold,0
6397,6397,6397,2723,SDG-16 --- This article examines the neo-conse...,16,neutral,neutral,neutral,negative,negative,Silver,0
6398,6398,6398,6710,SDG-16 --- In Phillips Petroleum Co. v. Shutts...,16,neutral,neutral,neutral,positive,neutral,Gold,0


In [ ]:
# train_df = pd.read_excel(input_data_path_train)
# train_df.head(3)

In [ ]:
# test_df = pd.read_excel(input_data_path)
# test_df.head(3)

In [ ]:
X_train = list(train_df.text.values)
y_train = list(train_df.label.values)

X_test = list(test_df.text.values)
y_test_id = list(test_df.text_id.values)
y_test = list(test_df.label.values)

In [ ]:
len(X_train)

5824

In [ ]:
len(y_train)

5824

In [ ]:
num_classes = 3

In [ ]:
# print("input_ids.shape =", input_ids.shape)
# print("labels.shape    =", labels.shape)

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Supponiamo che tu abbia già i dati di addestramento e di test
# X_train, y_train, X_test, y_test sono liste di testi e le rispettive etichette

# Carica il tokenizer e il modello BERT preaddestrato
tokenizer = BertTokenizer.from_pretrained('nbroad/ESG-BERT')
model = BertForSequenceClassification.from_pretrained('nbroad/ESG-BERT', num_labels=num_classes, ignore_mismatched_sizes=True)

# Tokenizza i dati di addestramento e di test
tokenized_train = tokenizer(X_train, truncation=True, padding=True, max_length=128, return_tensors='pt')
tokenized_test = tokenizer(X_test, truncation=True, padding=True, max_length=128, return_tensors='pt')

# Creazione dei DataLoader
train_dataset = TensorDataset(tokenized_train['input_ids'], tokenized_train['attention_mask'], torch.tensor(y_train))
test_dataset = TensorDataset(tokenized_test['input_ids'], tokenized_test['attention_mask'], torch.tensor(y_test))

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# Impostazione del dispositivo di esecuzione (CPU o GPU, se disponibile)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Impostazione dell'ottimizzatore e della funzione di perdita
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
criterion = torch.nn.CrossEntropyLoss()

# Addestramento del modello
num_epochs = 5
# for epoch in range(num_epochs):
#     model.train()
#     for input_ids, attention_mask, labels in train_loader:
#         input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

#         optimizer.zero_grad()
#         outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
#         loss = outputs.loss
#         loss.backward()
#         optimizer.step()

# for epoch in range(num_epochs):
#     model.train()
#     epoch_loss = 0.0
#     progress_bar = tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}', leave=False, dynamic_ncols=True)

#     for input_ids, attention_mask, labels in progress_bar:
#         input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

#         optimizer.zero_grad()
#         outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
#         loss = outputs.loss
#         loss.backward()
#         optimizer.step()

#         epoch_loss += loss.item()
#         progress_bar.set_postfix(loss=epoch_loss / len(train_loader), refresh=True)

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0.0
    progress_bar = tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}', leave=False, dynamic_ncols=True)

    for input_ids, attention_mask, labels in progress_bar:
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        progress_bar.set_postfix(loss=epoch_loss / len(train_loader), refresh=True)

    # Valuta il modello alla fine di ogni epoch
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for input_ids, attention_mask, labels in test_loader:
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1).cpu().numpy()

            all_preds.extend(preds)
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    print(f'Epoch {epoch + 1}/{num_epochs} - Loss: {epoch_loss / len(train_loader):.4f} - Accuracy: {accuracy:.4f}')

    # Riporta il modello in modalità di addestramento
    model.train()

# Valutazione del modello
model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for input_ids, attention_mask, labels in test_loader:
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1).cpu().numpy()

        all_preds.extend(preds)
        all_labels.extend(labels.cpu().numpy())


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at nbroad/ESG-BERT and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([26]) in the checkpoint and torch.Size([3]) in the model instantiated
- classifier.weight: found shape torch.Size([26, 768]) in the checkpoint and torch.Size([3, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to u

Epoch 1/5 - Loss: 0.7556 - Accuracy: 0.5816


Epoch 2/5 - Loss: 0.4819 - Accuracy: 0.5451


Epoch 3/5 - Loss: 0.2431 - Accuracy: 0.5399


Epoch 4/5 - Loss: 0.1247 - Accuracy: 0.5278


Epoch 5/5 - Loss: 0.0939 - Accuracy: 0.5781


In [ ]:
# Calcolo delle metriche di valutazione
print(len(all_labels))
print(len(all_preds))
print(classification_report(all_labels, all_preds, digits=4))

576
576
              precision    recall  f1-score   support

           0     0.5322    0.5662    0.5487       219
           1     0.5975    0.6545    0.6247       220
           2     0.6373    0.4745    0.5439       137

    accuracy                         0.5781       576
   macro avg     0.5890    0.5651    0.5724       576
weighted avg     0.5821    0.5781    0.5766       576



In [ ]:
all_labels[:5]

[1, 0, 1, 1, 0]

In [ ]:
all_preds[:5]

[1, 0, 0, 1, 0]

In [ ]:
y_test_id[:5]

[210, 251, 441, 591, 1250]

In [ ]:
df_final = pd.DataFrame({
    'text_id': y_test_id,
    'predicted_label': all_preds,
    'true_label': all_labels
})

In [ ]:
text_id_order = [210,
2715,
4478,
496,
2768,
5812,
1677,
4653,
928,
6619,
3368,
4369,
378,
4988,
4578,
2445,
3164,
2936,
3074,
4113,
6044,
3317,
1198,
3679,
3958,
2155,
6174,
5429,
826,
2550,
3863,
1231,
3990,
2178,
2636,
6318,
1555,
4565,
6637,
2510,
5038,
2139,
1610,
5475,
412,
6119,
1197,
5468,
5353,
5057,
5998,
117,
3987,
6421,
1422,
3854,
1372,
4217,
2151,
1827,
4834,
3393,
1084,
1765,
4480,
3293,
2144,
3089,
6082,
6656,
2530,
809,
3830,
2930,
4863,
5788,
1841,
732,
2039,
4150,
1567,
2996,
6230,
4325,
729,
2596,
3050,
240,
5933,
2254,
2141,
1882,
3665,
4033,
651,
3011
]

In [ ]:
# 1. Crea un dizionario che mappa i text_id alla loro posizione (indice)
position_map = {tid: int(i) for i, tid in enumerate(text_id_order)}
position_map

{210: 0,
 2715: 1,
 4478: 2,
 496: 3,
 2768: 4,
 5812: 5,
 1677: 6,
 4653: 7,
 928: 8,
 6619: 9,
 3368: 10,
 4369: 11,
 378: 12,
 4988: 13,
 4578: 14,
 2445: 15,
 3164: 16,
 2936: 17,
 3074: 18,
 4113: 19,
 6044: 20,
 3317: 21,
 1198: 22,
 3679: 23,
 3958: 24,
 2155: 25,
 6174: 26,
 5429: 27,
 826: 28,
 2550: 29,
 3863: 30,
 1231: 31,
 3990: 32,
 2178: 33,
 2636: 34,
 6318: 35,
 1555: 36,
 4565: 37,
 6637: 38,
 2510: 39,
 5038: 40,
 2139: 41,
 1610: 42,
 5475: 43,
 412: 44,
 6119: 45,
 1197: 46,
 5468: 47,
 5353: 48,
 5057: 49,
 5998: 50,
 117: 51,
 3987: 52,
 6421: 53,
 1422: 54,
 3854: 55,
 1372: 56,
 4217: 57,
 2151: 58,
 1827: 59,
 4834: 60,
 3393: 61,
 1084: 62,
 1765: 63,
 4480: 64,
 3293: 65,
 2144: 66,
 3089: 67,
 6082: 68,
 6656: 69,
 2530: 70,
 809: 71,
 3830: 72,
 2930: 73,
 4863: 74,
 5788: 75,
 1841: 76,
 732: 77,
 2039: 78,
 4150: 79,
 1567: 80,
 2996: 81,
 6230: 82,
 4325: 83,
 729: 84,
 2596: 85,
 3050: 86,
 240: 87,
 5933: 88,
 2254: 89,
 2141: 90,
 1882: 91,
 3665: 92

In [ ]:
# 2. Aggiungi una colonna di supporto (sort_key) basata su position_map
#    Se alcuni text_id non fossero presenti in position_map, il risultato
#    di map sarebbe NaN. Puoi decidere come gestirli (ad esempio, dandogli
#    un valore alto se vuoi metterli in fondo).
df_final['sort_key'] = df_final['text_id'].map(position_map)
df_final_test_sel = df_final[~df_final.sort_key.isna()]
df_final_test_sel['sort_key'] = df_final_test_sel['sort_key'].astype(int)
df_final_test_sel

<ipython-input-23-d1148eb0862f>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final_test_sel['sort_key'] = df_final_test_sel['sort_key'].astype(int)


,text_id,predicted_label,true_label,sort_key
0,210,1,1,0
6,2715,1,1,1
16,4478,0,0,2
21,5057,1,1,49
25,5353,0,2,48
...,...,...,...,...
545,1197,0,0,46
552,3011,0,0,95
555,4033,0,0,93
566,5468,0,0,47


In [ ]:
# 3. Ordina il DataFrame secondo la colonna di supporto
df_final_test_sel_sorted = df_final_test_sel.sort_values('sort_key', na_position='last')
df_final_test_sel_sorted

,text_id,predicted_label,true_label,sort_key
0,210,1,1,0
6,2715,1,1,1
16,4478,0,0,2
40,496,0,0,3
53,2768,0,1,4
...,...,...,...,...
513,1882,1,1,91
526,3665,1,1,92
555,4033,0,0,93
543,651,1,1,94


In [ ]:
df_final_test_sel_sorted.to_excel(base_path + "/../AnalisiTesti/data/EtichettaturaManuale-DatasetSDG/BERTSDG_predictions_on_test_dataset.xlsx")